# Description:

In [ ]:
#Description: This program detects if an email is spam (1) or not (0).

#Import libaries


In [1]:
import numpy as np
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import string 

#Load Data

In [ ]:
#Load Data:
#Opens an upload-Pop_up-Window

#from google.colab import files 
#uploaded = files.upload()

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#previous upload method: 
#df = pd.read_csv("/content/gdrive/MyDrive/dev/Womens Clothing E-Commerce Reviews.csv")

#New method 
#Download directly from my Gdrive
#uploaded = files.download("/content/gdrive/MyDrive/dev/Email-spam-detection-master/emails.csv")

#Not working file over 8MB

In [3]:
#file = '/content/gdrive/MyDrive/dev/Email-spam-detection-master/emails.csv'
file = '/content/gdrive/MyDrive/dev/Emails_analysis_labeled_utf8.CSV'
df = pd.read_csv(file)
df

#import io
#df2 = pd.read_csv(io.BytesIO(uploaded['Filename.csv']))
# Dataset is now stored in a Pandas Dataframe

,Betreff,Text,Von: (Name),Von: (Adresse),Von: (Typ),An: (Name),An: (Adresse),An: (Typ),CC: (Name),CC: (Adresse),CC: (Typ),BCC: (Name),BCC: (Adresse),BCC: (Typ),Abrechnuengsinformationen,Kategorien,Reisekilometer,Vertrauelichkeit,Wichtigkeit,Spam,Tags
0,"JF ""Deep dive"" GEHE AKADEMIE/ Sylvenstein - Of...",Besprechueng offener Fragen/ weiterer Verlauef...,Piehl Thorsten,Thorsten.Piehl@gehe.de,SMTP,Schneil Goenna;Dittor Maria;Kraiser Iris;Meint...,Goenna.Schneil@gehe.de;Maria.Dittor@gehe.de;Ir...,SMTP;SMTP;SMTP;SMTP;SMTP;SMTP;SMTP;SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,Normal,1,['Spam']
1,Re: NEueER Banner Lifescan // GEHE AKADEMIE //...,"Gueten Morgen Daniela,\r\nbin wieder zuerueck ...",Chris Guenter,cg@wacg.de,SMTP,"Miksch, Daniela",Daniela.Miksch@alliance-healthcare.de,SMTP,"Dittor, Maria;Piehl, Thorsten;pp@wacg.de;info@...",maria.dittor@gehe.de;Thorsten.Piehl@gehe.de;pp...,SMTP;SMTP;SMTP;SMTP,NaN,NaN,NaN,NaN,NaN,NaN,Normal,Normal,1,['Spam']
2,Re: Bannerplatzierueng Ende Aueguest // Anfang...,"Schoenen gueten Morgen zuesammen, hi Maria,\r\...",Chris Guenter,cg@wacg.de,SMTP,Pauela Pantleon,pp@wacg.de,SMTP,Dittor Maria;info@sylvenstein.com;Kraiser Iris...,Maria.Dittor@gehe.de;info@sylvenstein.com;Iris...,SMTP;SMTP;SMTP;SMTP,NaN,NaN,NaN,NaN,NaN,NaN,Normal,Normal,1,"['Spam', 'Apps']"
3,AW: NEueER Banner Lifescan // GEHE AKADEMIE //...,"Hallo zuesammen, \r\n\r\n \r\n\r\nggf. koennte...","Miksch, Daniela",Daniela.Miksch@alliance-healthcare.de,SMTP,"Dittor, Maria;Piehl, Thorsten;'Chris Guenter';...",maria.dittor@gehe.de;Thorsten.Piehl@gehe.de;cg...,SMTP;SMTP;SMTP;SMTP;SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,Hoch,0,"['Gehe', 'Banner']"
4,WG: [EXTERNAL] Reportings,"Liebes Sylvenstein Team,\r\n\r\n \r\n\r\nbitte...",Schneil Goenna,Goenna.Schneil@gehe.de,SMTP,'info@sylvenstein.com',info@sylvenstein.com,SMTP,Schneil Goenna;Dittor Maria,Goenna.Schneil@gehe.de;Maria.Dittor@gehe.de,SMTP;SMTP,NaN,NaN,NaN,NaN,NaN,NaN,Normal,Hoch,0,"['Gehe', 'Banner']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,[Sylvenstein SCRueM Team 1 - Auefgabe #672] (I...,"Sehr geehrtes Projektmitglied,\r\n\r\ndie Proj...",info@sylvenstein.com,info@sylvenstein.com,SMTP,info@sylvenstein.com,info@sylvenstein.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,Normal,1,"['Spam', 'Fake']"
134,[Sylvenstein SCRueM Team 1 - Auefgabe #671] (N...,"Sehr geehrtes Projektmitglied,\r\n\r\ndie Proj...",info@sylvenstein.com,info@sylvenstein.com,SMTP,info@sylvenstein.com,info@sylvenstein.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,Normal,1,['Spam']
135,[Sylvenstein SCRueM Team 1 - Auefgabe #670] (N...,"Sehr geehrtes Projektmitglied,\r\n\r\ndie Proj...",info@sylvenstein.com,info@sylvenstein.com,SMTP,info@sylvenstein.com,info@sylvenstein.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,Normal,0,"['Gehe', 'elearning']"
136,[Sylvenstein SCRueM Team 1 - Auefgabe #669] (N...,"Sehr geehrtes Projektmitglied,\r\n\r\ndie Proj...",info@sylvenstein.com,info@sylvenstein.com,SMTP,info@sylvenstein.com,info@sylvenstein.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,Normal,0,"['Gehe', 'elearning']"


In [4]:
#Print the shape (Get the number of rows and cols)
#Before checking for duplicates or empty rows
df.shape

(138, 21)

In [5]:
#Get the column names
df.columns

Index(['Betreff', 'Text', 'Von: (Name)', 'Von: (Adresse)', 'Von: (Typ)',
       'An: (Name)', 'An: (Adresse)', 'An: (Typ)', 'CC: (Name)',
       'CC: (Adresse)', 'CC: (Typ)', 'BCC: (Name)', 'BCC: (Adresse)',
       'BCC: (Typ)', 'Abrechnuengsinformationen', 'Kategorien',
       'Reisekilometer', 'Vertrauelichkeit', 'Wichtigkeit', 'Spam', 'Tags'],
      dtype='object')

In [6]:
#Checking for duplicates and removing them
df.drop_duplicates(inplace = True)

In [7]:
#Show the new shape (number of rows & columns)
df.shape

(138, 21)

In [8]:
#Show the number of missing (NAN, NaN, na) data for each column
df.isnull().sum()

Betreff                        0
Text                           1
Von: (Name)                    0
Von: (Adresse)                 0
Von: (Typ)                     0
An: (Name)                     5
An: (Adresse)                  5
An: (Typ)                      5
CC: (Name)                    84
CC: (Adresse)                 84
CC: (Typ)                     84
BCC: (Name)                  138
BCC: (Adresse)               138
BCC: (Typ)                   138
Abrechnuengsinformationen    138
Kategorien                   138
Reisekilometer               138
Vertrauelichkeit               1
Wichtigkeit                    0
Spam                           0
Tags                           0
dtype: int64

In [9]:
#Need to download stopwords-Package
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
# Function:
# Tokenization (a list of tokens), will be used as the analyzer
# 1.Punctuations are [!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]
# 2.Stop words in natural language processing, are useless words (data).
# 3 Return a list of clean words

def process_text(text):
    
    #1 Remove Punctuationa
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    #2 Remove Stop Words
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    #3 Return a list of clean words
    return clean_words

In [12]:
#Show the Tokenization (a list of tokens )
df['text'].head().apply(process_text)

0    [Besprechueng, offener, Fragen, weiterer, Verl...
1    [Gueten, Morgen, Daniela, bin, wieder, zueruec...
2    [Schoenen, gueten, Morgen, zuesammen, hi, Mari...
3    [Hallo, zuesammen, ggf, koennte, es, zue, Verw...
4    [Liebes, Sylvenstein, Team, bitte, senden, Sie...
Name: Text, dtype: object

In [ ]:
#Show the Tokenization (a list of tokens )
df['Text'].head().apply(process_text)

In [13]:
#Example 
message4 = 'hello World hello world play'
message5 = 'test test test one hello'

print(message4)
print()

#Convert the text to a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer
bow4 = CountVectorizer(analyzer=process_text).fit_transform([[message4],[message5]])
print(bow4)
print()

print(bow4.shape)

hello World hello world play

  (0, 1)	2
  (0, 0)	1
  (0, 5)	1
  (0, 3)	1
  (1, 1)	1
  (1, 4)	3
  (1, 2)	1

(2, 6)


In [ ]:
#Convert the text into a matrix of token counts.
from sklearn.feature_extraction.text import CountVectorizer
messages_bow = CountVectorizer(analyzer=process_text).fit_transform(df['text'])

In [ ]:
#Split data into 80% training & 20% testing data sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(messages_bow, df['spam'], test_size = 0.20, random_state = 0)

In [ ]:
#Get the shape of messages_bow
messages_bow.shape

(5695, 37229)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
#Print the predictions
print(classifier.predict(X_train))

#Print the actual values
print(y_train.values)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [ ]:
#Evaluate the model on the training data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_train)

print(classification_report(y_train ,pred ))
print('Confusion Matrix: \n',confusion_matrix(y_train,pred))
print()
print('Accuracy: ', accuracy_score(y_train,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3457
           1       0.99      1.00      0.99      1099

    accuracy                           1.00      4556
   macro avg       0.99      1.00      1.00      4556
weighted avg       1.00      1.00      1.00      4556

Confusion Matrix: 
 [[3445   12]
 [   1 1098]]

Accuracy:  0.9971466198419666


In [ ]:
#Print the predictions
print('Predicted value: ',classifier.predict(X_test))

#Print Actual Label
print('Actual value: ',y_test.values)

Predicted value:  [1 0 0 ... 0 0 0]
Actual value:  [1 0 0 ... 0 0 0]


In [ ]:
#Evaluate the model on the test data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_test)

print(classification_report(y_test ,pred ))
print('Confusion Matrix: \n', confusion_matrix(y_test,pred))
print()
print('Accuracy: ', accuracy_score(y_test,pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       870
           1       0.97      1.00      0.98       269

    accuracy                           0.99      1139
   macro avg       0.98      0.99      0.99      1139
weighted avg       0.99      0.99      0.99      1139

Confusion Matrix: 
 [[862   8]
 [  1 268]]

Accuracy:  0.9920983318700615


## own prediction 